# Artificial Neural Network

### Importing the libraries

In [52]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [53]:
# prints the version of tensorflow
tf.__version__

'2.20.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [54]:
dataset = pd.read_csv('Churn_Modelling.csv')
# use all rows from index 3 to one index before the last index
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [55]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [56]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [57]:
from sklearn.preprocessing import LabelEncoder
# male and female habe a relation, therefore the LabelEncoder is suitable
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [58]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# [1]: the index of the column which the one hot encoding will be applied
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
# I use one-hot encoding here since countries have no relationship.
X = np.array(ct.fit_transform(X))

In [60]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [62]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit_transform: train and apply
X_train = sc.fit_transform(X_train)
# only apply because it don't need training
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [63]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [64]:
# first hidden layer, units: number of neurons for this layer
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer (will be deep)

In [65]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [66]:
# for classification output there should be a one hot endcoding output e.g. countries spain, french, germany. Therefore the output layer will have 3 neurons (units) -> for spain 1:0:0, french 0:1:0, germany 0:0:1
# sigmoid: get the probability between the range of 0 to 1. Note: for a categorial output the activation function for the output layer should be soft_max

ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN (Artificial Neural Network)

### Compiling the ANN

In [67]:
# optimizer: kind of gradient descent algorithm you want to use. Adam is ein example for a gradient descent algorithm
# loss: kind of loss function to use. binary_crossentropy is used for a binary output.
# metrics: all metrics will be calculated after each training epoch. After the training you can analyze how the metrics changes after each epochs.
# Accuracy Metric: Number of correct predictions between 0 and 1.
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

### Training the ANN on the Training set

In [68]:
print(X_train)

[[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]


In [69]:
print(y_train)

[0 0 0 ... 0 0 1]


In [70]:
# batch: a part of the whole dataset for example: 30 data points of 10'000
# batch_size = 32: The whole dataset will be splitted in subsets of 32 datapoints per subset. The weight will be only changed when the 32 datapoints have predicted their output variable.
# epochs: Number of iterations for training the whole dataset. E.g. 100 means the whole dataset of 10'000 rows will be trained 100 times. Each epoch is splitted in batches.
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6374 - loss: 0.6383
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8001 - loss: 0.4707
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8110 - loss: 0.4287
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8154 - loss: 0.4130
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8175 - loss: 0.4040
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8234 - loss: 0.3951  
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8384 - loss: 0.3825  
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8465 - loss: 0.3720  
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.8520 - loss: 0.3644
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8553 - loss: 0.3591  
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8566 - loss: 0.3553
Epoch 12/100
250/250 ━━━━━━━━

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [71]:
X_prediction = [[1, 0 , 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]
X_prediction_scaled = sc.transform(X_prediction)
print(ann.predict(X_prediction_scaled) > 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [72]:
y_pred = ann.predict(X_test) > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [73]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1514   81]
 [ 203  202]]


0.858